# Recurrent Neural Networks
The following Recurrent Neural Networks (RNN) will be used for musical genre classification. This is because the task of classify all the new music that is released nowadays it is impossible to be done by a human being.

## Initialization

### Libraries

In [2]:
# Import general purpose python libraries
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.layers import LSTM, Bidirectional, Dense, Attention,Input, Dropout, TimeDistributed
from tensorflow.keras.models import Model, Sequential
from keras.optimizers import SGD, Adam
from keras.utils import image_dataset_from_directory

# Import function to plot the results
import plots

2024-01-13 21:19:28.771740: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Data Configuration Parameters
Configuration variables related to the data

In [4]:
# Randomize the initial network weights
random_seed = True

# Paths to where training, testing, and validation images are
database_dir = 'dataset'
train_dir = f'{database_dir}/training/spectrogram'
val_dir = f'{database_dir}/val/spectrogram'
test_dir = f'{database_dir}/test/spectrogram'

# Directory where to store weights of the model and results
root_dir = "results"
# Create root directory for results if it does not exist
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Input dimension (number of subjects in our problem)
num_classes = 6

# Name of each gesture of the database
CLASSES = [x for x in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, x))]
print(f'The classess to classify are: {CLASSES}')

# Parameters that characterise the spectrogram
img_height = 369
img_width = 496
img_channels = 3
color_mode = 'rgb'

The classess to classify are: ['Alternative', 'Pop', 'Classical', 'Rock', 'Dance', 'Techno']


### Configuration Training Parameters

In [5]:
# Parameters that configures the training process
batch_size = 1  # Batch size
epochs = 5  # Number of epochs
initial_lr = 1e-5   # Learning rate
seed = 42  # Random number
modelRNN = 'LSTM'  # RNN model which will be used
version = f'BS{batch_size}_E{epochs}_LR{initial_lr}'
experiment_dir = f'{root_dir}/{modelRNN}'

# Create experiment directory if it does not exist
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

# Set random seed
if random_seed:
    seed = np.random.randint(0,2*31-1)
else:
    seed = 5
np.random.seed(seed)
tf.random.set_seed(seed)

### Loading of training, validation and test datasets of images
1.   Training dataset
2.   Validation dataset
3.   Test dataset

In [ ]:
# 1. Generate train dataset (ds) from directory of samples
train_ds = image_dataset_from_directory(directory=train_dir,
                                        label_mode = 'int',
                                        class_names=CLASSES,
                                        batch_size=batch_size,
                                        color_mode=color_mode,
                                        image_size=(img_width,img_height), shuffle=True)

# 2. Generate validation dataset (ds) from directory of samples
val_ds  = image_dataset_from_directory(directory=val_dir,
                                       label_mode = 'int',
                                       class_names=CLASSES,
                                       batch_size=batch_size,
                                       color_mode=color_mode,
                                       image_size=(img_width,img_height), shuffle=True)

# 3. Generate test dataset (ds) from directory of samples
test_ds = image_dataset_from_directory(directory=test_dir,
                                       label_mode = 'int',
                                       class_names=CLASSES,
                                       batch_size=batch_size,
                                       color_mode=color_mode,
                                       image_size=(img_width,img_height),
                                       shuffle = False)

train_ds = train_ds.map(lambda x, y: (tf.squeeze(x, axis=0), y))
val_ds = val_ds.map(lambda x, y: (tf.squeeze(x, axis=0), y))
test_ds = test_ds.map(lambda x, y: (tf.squeeze(x, axis=0), y))
print(train_ds.element_spec)
print(val_ds.element_spec)
print(test_ds.element_spec)

Found 1186 files belonging to 6 classes.
Found 204 files belonging to 6 classes.
Found 204 files belonging to 6 classes.


2024-01-13 21:19:52.466107: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..

(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>


...

(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
(496, 369, 3)
<dtype: 'float32'>
<dtype: 'int32'>
Error in callback <function flush_figures at 0x7f9b8916e280> (for post_execute):


# Training process
#### Available Models: LSTM & BiLSTM

### Long Short-Term Memory (LSTM)

In [72]:
def lstm(height, width, out_dim):
    # Sequential Model
    model_lstm = Sequential()
    model_lstm.add(LSTM(128, input_shape=(width, height), activation='relu', return_sequences=True))
    model_lstm.add(Dropout(0.2))

    #model_lstm.add(Attention(use_scale=True))

    model_lstm.add(LSTM(128, activation='relu'))
    model_lstm.add(Dropout(0.2))

    model_lstm.add(Dense(32, activation='relu'))
    model_lstm.add(Dropout(0.2))

    model_lstm.add(Dense(6, activation='softmax'))

    return model_lstm
    # # Input layer
    # input_layer = Input(shape=(height, width))
    # 
    # # First LSTM layer
    # lstm_layer = TimeDistributed(LSTM(256, activation='tanh', return_sequences=True))(input_layer)
    # 
    # # First Attention layer
    # attention_layer = Attention(use_scale=True)([lstm_layer, lstm_layer])
    # 
    # # First Dropout layer for regularization
    # dropout_layer = Dropout(0.2)(attention_layer)
    # 
    # # First Dense layer
    # dense_layer = TimeDistributed(Dense(uints=out_dim, activation='relu'))(dropout_layer)
    # 
    # # Second LSTM layer
    # lstm_layer_2 = LSTM(128, activation='tanh', return_sequences=True)(dense_layer)
    # 
    # # Second Attention layer
    # attention_layer_2 = Attention(use_scale=True)([lstm_layer_2, lstm_layer_2])
    # 
    # # Second Dense layer
    # output_layer = Dense(units=out_dim, activation='softmax')(attention_layer_2)
    # 
    # # Final LSTM model
    # model_lstm = Model(inputs=input_layer, outputs=output_layer)
    # 
    # return model_lstm

### Bidirectional Long Short-Term Memory (BiLSTM)

In [70]:
def bi_lstm(height, width, out_dim):
    # Input layer
    input_layer = Input(shape=(height, width))

    # First BiLSTM layer
    bilstm_layer = TimeDistributed(Bidirectional(LSTM(out_dim, activation='tanh', return_sequences=True)))(input_layer)

    # First Attention layer
    attention_layer = Attention(use_scale=True)([bilstm_layer, bilstm_layer])

    # First Dropout layer for regularization
    dropout_layer = Dropout(0.2)(attention_layer)

    # First Dense layer
    dense_layer = TimeDistributed(Dense(units=out_dim, activation='relu'))(dropout_layer)

    # Second BiLSTM layer
    bilstm_layer_2 = Bidirectional(LSTM(out_dim, activation='tanh', return_sequences=True))(dense_layer)

    # Second Attention layer
    attention_layer_2 = Attention(use_scale=True)([bilstm_layer_2, bilstm_layer_2])

    # Second Dense layer
    output_layer = Dense(units=out_dim, activation='softmax')(attention_layer_2)

    # Final LSTM model
    model_bilstm = Model(inputs=input_layer, outputs=output_layer)
    
    return model_bilstm

## Model execution

In [1]:
#Model
model = None
if modelRNN == 'LSTM':
    model = lstm(img_height, img_width, num_classes)
elif modelRNN == 'BiLSTM':
    model = bi_lstm(img_height, img_width, num_classes)
else:
    print('Wrong model selection or Model no available\n')

# Print the architecture of the model
model.summary()

NameError: name 'modelRNN' is not defined

## Set model training process
#### Configuration of several training decisions:
1. Optimizer using `Adam`
2. Model training configuration using `compile` with `categorical_crossentropy` due to the classification labeling

In [58]:
# 1. Configure optimizer
adam = Adam(learning_rate=initial_lr)

# 2. Configure training process
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])


AttributeError: 'NoneType' object has no attribute 'compile'

## Train the model
1. Load parameters from previous trainings if they exist.
2. Fit the model
3. Save the weights

In [ ]:
# Load pretrained model
weights_path = f"weights_{version}.h5" # Name of the file to store the weights
weights_file = Path(weights_path)
weights_load_path = f'{experiment_dir}/{weights_path}'
if weights_load_path:
    try:
        model.load_weights(weights_load_path)
        print("Loaded model from {}".format(weights_load_path))
    except:
        print("Impossible to find weight path. Returning untrained model")

# Fit the model
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, batch_size=batch_size)

# Save weights
weights_save_path = os.path.join(experiment_dir, weights_path)
model.save_weights(weights_save_path)

Impossible to find weight path. Returning untrained model
Epoch 1/5


## Training Results
Accuracy and Loss obtained along the training process

In [ ]:
plots.accloss(history, modelRNN, experiment_dir, version)

# Testing
### Model Testing
1. Compute the loss function and accuracy for the test data
2. Confusion Matrix obtained from testing results

In [ ]:
# Evaluate model
scores = model.evaluate(test_ds, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Loss: %.2f" % scores[0])

# Obtain results to present the confusion matrix
prob_class = model.predict(test_ds, batch_size=batch_size)
# Classified labels
y_pred = tf.argmax(prob_class, axis=-1)
# Ground truth
y_true = tf.argmax(tf.concat([label for image, label in test_ds], axis=0), axis=1)
# Visualize confusion matrix                                           
plots.cm(y_true, y_pred, modelRNN, CLASSES, experiment_dir, version)